# Fast Filtering with Large Option Panels: Implications for Asset Pricing

As we don't have access to the data used in the paper, we will simulate data from the option pricing models whown in the figures (page 59 environ) and then estimate the model given the artificial data.

In [ ]:
import matplotlib.pyplot as plt

import simulation

## 1. Simulating data

We will first simulate the data from the simple Stochastic Volatility model of Heston. The model is given by the following SDEs:

$$
\begin{aligned}
dS_t &= \mu S_t dt + \sqrt{V_t} S_t dW_t^S \\
dV_t &= \kappa (\theta - V_t) dt + \sigma \sqrt{V_t} dW_t^v
\end{aligned}
$$

where $W_t^S$ and $W_t^v$ are two correlated Brownian motions with correlation $\rho$. We will simulate the data for the following parameters:

In [ ]:
# Simulate
time, S, V = simulate_heston(S0, V0, mu, kappa, theta, sigma, rho, T, N, dt)

# Plot results
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, S, label="Stock Price")
plt.title("Heston Model Simulation")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.grid()
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, V, label="Variance", color="orange")
plt.xlabel("Time")
plt.ylabel("Variance")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

## 2. Boostrap Filter (Feynman-Kac) for the model under consideration

In this section, we will implement the boostrap filter for the model under consideration. The boostrap filter is a Monte Carlo method that allows to estimate the filtering distribution of the state variables given the observations. The algorithm is as follows:

1. Initialize the particles $x_0^{(i)}$ for $i=1,\ldots,N$ from the prior distribution $p(x_0)$.

2. For $t=1,\ldots,T$:
    1. For $i=1,\ldots,N$:
        1. Sample $x_t^{(i)}$ from the proposal distribution $q(x_t|x_{t-1}^{(i)},y_t)$.
        2. Compute the importance weight $w_t^{(i)} = \frac{p(y_t|x_t^{(i)}) p(x_t^{(i)}|x_{t-1}^{(i)})}{q(x_t^{(i)}|x_{t-1}^{(i)},y_t)}$.
    2. Normalize the weights $w_t^{(i)}$.
    3. Resample the particles $x_t^{(i)}$ with replacement according to the weights $w_t^{(i)}$.

3. Compute the estimate of the filtering distribution as $\hat{p}(x_t|y_{1:t}) = \sum_{i=1}^N w_t^{(i)} \delta_{x_t^{(i)}}(x_t)$.

We will implement the boostrap filter for the model under consideration.